## Pyspark 연습
* Basic Data handling
* Descriptive summary
* Feature extraction
* Build ML model

* 참고
    * https://spark.apache.org/docs/2.2.0/mllib-statistics.html
    * https://jaeyung1001.tistory.com/59
    * https://hendra-herviawan.github.io/pyspark-dataframe-row-columns.html
    * https://hendra-herviawan.github.io/pyspark-groupby-and-aggregate-functions.html

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StringType
import pandas as pd
from datetime import datetime, date
spark = SparkSession.builder.getOrCreate()
spark

In [2]:
df = spark.read.csv('./dataset/clickstream/total_datapoint.csv', inferSchema=True, header=True)

In [3]:
df.count()

8772446

In [4]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- CUS_ID: integer (nullable = true)
 |-- TIME_ID: integer (nullable = true)
 |-- SITE: string (nullable = true)
 |-- SITE_CNT: double (nullable = true)
 |-- ST_TIME: double (nullable = true)
 |-- SITE_NM: string (nullable = true)
 |-- BACT_NM: string (nullable = true)
 |-- MACT_NM: string (nullable = true)
 |-- ACT_NM: string (nullable = true)
 |-- AC_TIME: integer (nullable = true)
 |-- YY_MM_DD: integer (nullable = true)
 |-- DAY: integer (nullable = true)



In [5]:
df.show()

+---+------+----------+--------------------+--------+-------+-----------------------+-------------+------------+-------------+-------+--------+---+
|_c0|CUS_ID|   TIME_ID|                SITE|SITE_CNT|ST_TIME|                SITE_NM|      BACT_NM|     MACT_NM|       ACT_NM|AC_TIME|YY_MM_DD|DAY|
+---+------+----------+--------------------+--------+-------+-----------------------+-------------+------------+-------------+-------+--------+---+
|  0|     1|2012070905|    search.naver.com|     3.0|  794.0|            네이버 검색|인터넷/컴퓨터|        검색|     포털검색|      5|20120709|  0|
|  1|     1|2012072507|     plus.google.com|     1.0|    1.0|              구글 Plus|     커뮤니티|  블로그/SNS|          SNS|      7|20120725|  2|
|  2|     1|2012081116|joongang.joinsmsn...|     2.0|    5.0|               중앙일보|  뉴스/미디어|      일간지|   종합일간지|     16|20120811|  5|
|  3|     1|2012090304|      news.naver.com|     5.0|  504.0|            네이버 뉴스|  뉴스/미디어|  인터넷신문|     포털뉴스|      4|20120903|  0|
|  4|     1|2012090506|   

In [6]:
df = df.withColumn('TIME_ID', df['TIME_ID'].cast(StringType()))

In [7]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- CUS_ID: integer (nullable = true)
 |-- TIME_ID: string (nullable = true)
 |-- SITE: string (nullable = true)
 |-- SITE_CNT: double (nullable = true)
 |-- ST_TIME: double (nullable = true)
 |-- SITE_NM: string (nullable = true)
 |-- BACT_NM: string (nullable = true)
 |-- MACT_NM: string (nullable = true)
 |-- ACT_NM: string (nullable = true)
 |-- AC_TIME: integer (nullable = true)
 |-- YY_MM_DD: integer (nullable = true)
 |-- DAY: integer (nullable = true)



In [8]:
df = df.withColumn('new_date',F.to_date(F.unix_timestamp('TIME_ID', 'yyyyMMddHH').cast('timestamp')))

In [9]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- CUS_ID: integer (nullable = true)
 |-- TIME_ID: string (nullable = true)
 |-- SITE: string (nullable = true)
 |-- SITE_CNT: double (nullable = true)
 |-- ST_TIME: double (nullable = true)
 |-- SITE_NM: string (nullable = true)
 |-- BACT_NM: string (nullable = true)
 |-- MACT_NM: string (nullable = true)
 |-- ACT_NM: string (nullable = true)
 |-- AC_TIME: integer (nullable = true)
 |-- YY_MM_DD: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- new_date: date (nullable = true)



In [10]:
df.describe().show()

+-------+------------------+------------------+--------------------+-------------+------------------+------------------+-----------------+---------+-------+--------------------+-----------------+-------------------+------------------+
|summary|               _c0|            CUS_ID|             TIME_ID|         SITE|          SITE_CNT|           ST_TIME|          SITE_NM|  BACT_NM|MACT_NM|              ACT_NM|          AC_TIME|           YY_MM_DD|               DAY|
+-------+------------------+------------------+--------------------+-------------+------------------+------------------+-----------------+---------+-------+--------------------+-----------------+-------------------+------------------+
|  count|           8772446|           8772446|             8772446|      8772446|           8772446|           8772439|          8772446|  8772446|8772446|             8772446|          8772446|            8772446|           8772446|
|   mean|4066912.0803559236|1239.7318502730025|2.01252721501

In [27]:
df.describe(['SITE_CNT']).show()

+-------+------------------+
|summary|          SITE_CNT|
+-------+------------------+
|  count|           8772446|
|   mean| 5.478712094665502|
| stddev|11.282212254464055|
|    min|               1.0|
|    max|            1158.0|
+-------+------------------+



In [29]:
df.describe(['ST_TIME']).show()

+-------+------------------+
|summary|           ST_TIME|
+-------+------------------+
|  count|           8772439|
|   mean|134.85187802388822|
| stddev| 299.9778640844884|
|    min|               0.0|
|    max|           25274.0|
+-------+------------------+



In [11]:
df.groupBy('SITE').count().orderBy('count', ascending=False).show()

+-----------------+-------+
|             SITE|  count|
+-----------------+-------+
|    www.naver.com|1169663|
| search.naver.com| 611978|
|     www.daum.net| 547982|
|     www.nate.com| 283289|
|   blog.naver.com| 239893|
|  search.daum.net| 184816|
|   cafe.naver.com| 177760|
|   media.daum.net| 171983|
|   mail.naver.com| 157803|
|sstatic.naver.com| 153910|
|    kin.naver.com| 149198|
|   mail2.daum.net| 141135|
|    cafe.daum.net| 114749|
|   news.naver.com| 109648|
|   mail3.nate.com| 100717|
| www.facebook.com|  83023|
|   www.11st.co.kr|  80509|
|www.gmarket.co.kr|  78882|
|    news.nate.com|  76211|
|    map.naver.com|  70262|
+-----------------+-------+
only showing top 20 rows



In [12]:
df.select(F.countDistinct("SITE").alias("unique site")).show()

+-----------+
|unique site|
+-----------+
|      36911|
+-----------+



In [13]:
df.groupBy('SITE_NM').count().orderBy('count', ascending=False).show()

+---------------+-------+
|        SITE_NM|  count|
+---------------+-------+
|         네이버|1171030|
|    네이버 검색| 982165|
|           다음| 548509|
|         네이트| 283489|
|  네이버 블로그| 251179|
|      다음 검색| 208533|
|    네이버 카페| 182038|
|     미디어다음| 171983|
|      다음 메일| 163606|
|    네이버 메일| 161696|
|      다음 카페| 144828|
|    네이버 뉴스| 134934|
|    네이트 메일| 131739|
|    네이트 뉴스| 129788|
|          G마켓|  83757|
|       페이스북|  83268|
|  네이버 지식iN|  81095|
|         11번가|  80637|
|네이버 지식쇼핑|  62060|
|     KB국민은행|  53519|
+---------------+-------+
only showing top 20 rows



In [14]:
df.select(F.countDistinct("SITE_NM").alias("unique SITE_NM")).show()

+--------------+
|unique SITE_NM|
+--------------+
|         28876|
+--------------+



In [15]:
df.groupBy('BACT_NM').count().orderBy('count', ascending=False).show()

+--------------+-------+
|       BACT_NM|  count|
+--------------+-------+
| 인터넷/컴퓨터|4189770|
|   뉴스/미디어|1292845|
|          쇼핑|1128331|
|      커뮤니티| 975819|
|   금융/부동산| 317316|
|  엔터테인먼트| 205825|
|          게임| 124541|
| 비즈니스/경제|  87964|
|     정치/행정|  80489|
|    온라인교육|  60128|
|   정보통신/IT|  55450|
|        서비스|  46611|
|생활/가정/취미|  37293|
|          여행|  36060|
|     교육/학원|  29876|
|유통/판매/운송|  28359|
|          제조|  23227|
|     건강/의학|  17802|
|   스포츠/레저|  15501|
|사회/문화/종교|  12657|
+--------------+-------+
only showing top 20 rows



In [16]:
df.select(F.countDistinct("BACT_NM").alias("unique BACT_NM")).show()

+--------------+
|unique BACT_NM|
+--------------+
|            22|
+--------------+



In [17]:
df.groupBy('MACT_NM').count().orderBy('count', ascending=False).show()

+-----------------+-------+
|          MACT_NM|  count|
+-----------------+-------+
|             포털|2080020|
|             검색|1363454|
|       인터넷신문| 560634|
|         종합쇼핑| 520596|
|       블로그/SNS| 511496|
|     커뮤니케이션| 486011|
|     커뮤니티포털| 388400|
|         전문뉴스| 339706|
|           일간지| 288721|
|      의류 쇼핑몰| 266511|
|             은행| 137752|
|         다운로드| 113847|
|         가격비교| 103431|
|멀티미디어/동영상|  85128|
|             방송|  76139|
|       온라인게임|  60406|
|         웹서비스|  53473|
|             취업|  51231|
|   분야별커뮤니티|  49216|
|         쇼핑정보|  48528|
+-----------------+-------+
only showing top 20 rows



In [18]:
df.select(F.countDistinct("MACT_NM").alias("unique MACT_NM")).show()

+--------------+
|unique MACT_NM|
+--------------+
|           207|
+--------------+



In [26]:
df.groupBy('ACT_NM').count().orderBy('count', ascending=False).show(100)

+---------------------+-------+
|               ACT_NM|  count|
+---------------------+-------+
|             종합포털|2079071|
|             포털검색|1235966|
|             메일계정| 478821|
|             포털뉴스| 452782|
|         포털커뮤니티| 334921|
|           포털블로그| 304635|
|             오픈마켓| 255832|
|       여성의류쇼핑몰| 179458|
|           종합일간지| 166708|
|                  SNS| 141637|
|           종합쇼핑몰| 130269|
|             시중은행| 129963|
|           스포츠신문| 122358|
|             경제신문| 121289|
|           소셜커머스| 114148|
|       종합인터넷신문| 106116|
|         포털지식검색|  93483|
|             포털쇼핑|  69933|
|        동영상/비디오|  68634|
|      컨텐츠공유(P2P)|  56878|
|      위성/케이블채널|  51590|
|               IT뉴스|  47595|
|      연예/오락전문지|  44752|
|    포털지도/지역정보|  41401|
|             게임포털|  39564|
|             포털금융|  38440|
|           포털게시판|  34619|
|             신용카드|  34114|
|        외국신문/잡지|  34006|
|             포털만화|  31860|
|        종합구인/구직|  31336|
|             지역뉴스|  31069|
|             검색엔진|  29327|
|      

In [20]:
df.select(F.countDistinct("ACT_NM").alias("unique ACT_NM")).show()

+-------------+
|unique ACT_NM|
+-------------+
|         1216|
+-------------+



## GROUPING 을 통한 Feature 생성하기
* PAGEVIEW
* DURATION
* 시간대별 PAGEVIEW / DURATION 
* 요일별 PAGEVIEW / DURATION 
* BACT_NM duration 합
* Top MACT_NM 50 duration 합
* Top ACT_NM 500 duration 합 

In [21]:
df.columns

['_c0',
 'CUS_ID',
 'TIME_ID',
 'SITE',
 'SITE_CNT',
 'ST_TIME',
 'SITE_NM',
 'BACT_NM',
 'MACT_NM',
 'ACT_NM',
 'AC_TIME',
 'YY_MM_DD',
 'DAY',
 'new_date']

In [30]:
# user pageview
pageview_sum = df.groupby("CUS_ID").agg(F.sum("SITE_CNT").alias("pg_sum"))
pageview_avg = df.groupby("CUS_ID").agg(F.avg("SITE_CNT").alias("pg_avg"))

In [27]:
# user total duration
duration_sum = df.groupby("CUS_ID").agg(F.sum("ST_TIME").alias("pg_sum"))
duration_avg = df.groupby("CUS_ID").agg(F.avg("ST_TIME").alias("pg_sum"))

In [34]:
# pageview / duration (sum) by time
timeby_pg = df.groupby(["CUS_ID"]).pivot("AC_TIME").agg(F.sum("SITE_CNT"))
timeby_dur = df.groupby(["CUS_ID"]).pivot("AC_TIME").agg(F.sum("ST_TIME"))

In [35]:
# pageview / duration (sum) by day
dayby_pg = df.groupby(["CUS_ID"]).pivot("DAY").agg(F.sum("SITE_CNT"))
dayby_dur = df.groupby(["CUS_ID"]).pivot("DAY").agg(F.sum("ST_TIME"))

In [48]:
bact_dur = df.groupby(["CUS_ID"]).pivot("BACT_NM").agg(F.sum("ST_TIME"))
bact_dur.show()

+------+---------+-------+---------+-----------+-----------+---------+-------------+--------------+--------------+-------+--------+-----------+------------+------+----------+--------------+-------------+-----------+---------+------+--------+-----+
|CUS_ID|건강/의학|   게임|교육/학원|금융/부동산|뉴스/미디어|문학/예술|비즈니스/경제|사회/문화/종교|생활/가정/취미| 서비스|    쇼핑|스포츠/레저|엔터테인먼트|  여행|온라인교육|유통/판매/운송|인터넷/컴퓨터|정보통신/IT|정치/행정|  제조|커뮤니티| 학문|
+------+---------+-------+---------+-----------+-----------+---------+-------------+--------------+--------------+-------+--------+-----------+------------+------+----------+--------------+-------------+-----------+---------+------+--------+-----+
|  2366|     null|   null|   4057.0|      312.0|     7190.0|     null|         23.0|        1880.0|         130.0|  171.0| 15619.0|       92.0|      3999.0|  57.0|   71405.0|          83.0|     119657.0|      334.0|   1367.0| 134.0| 20106.0| null|
|  1580|     77.0|   11.0|     67.0|    74965.0|   113190.0|     null|        735.0|         119.0|  

In [50]:
tmp_list = df.groupBy('MACT_NM').count().orderBy('count', ascending=False).select('MACT_NM').collect()
top50_mact = [row.MACT_NM for row in tmp_list][:50]
top50_mact

['포털',
 '검색',
 '인터넷신문',
 '종합쇼핑',
 '블로그/SNS',
 '커뮤니케이션',
 '커뮤니티포털',
 '전문뉴스',
 '일간지',
 '의류 쇼핑몰',
 '은행',
 '다운로드',
 '가격비교',
 '멀티미디어/동영상',
 '방송',
 '온라인게임',
 '웹서비스',
 '취업',
 '분야별커뮤니티',
 '쇼핑정보',
 '게임포털',
 '지불/결제',
 '생활',
 '부동산',
 '금융',
 '모바일컨텐츠',
 '증권/주식',
 '영화',
 '솔루션',
 '만화/애니메이션',
 '공공서비스',
 '학술정보',
 '도서/음반/악기 쇼핑몰',
 'SaaS/ASP',
 '정부/기관',
 '패션잡화 쇼핑몰',
 '음악',
 '취미/스포츠 쇼핑몰',
 '무선/이동통신',
 '자동차',
 '운송',
 '생활용품 쇼핑몰',
 '여행사',
 '도메인/호스팅',
 '화장품/미용 쇼핑몰',
 '어린이커뮤니티',
 '통신사',
 '쇼핑기타',
 '학생/교과교육',
 '도박']

In [51]:
mact_dur = df.groupby(["CUS_ID"]).pivot("MACT_NM", top50_mact).agg(F.sum("ST_TIME"))

In [52]:
tmp_list = df.groupBy('ACT_NM').count().orderBy('count', ascending=False).select('ACT_NM').collect()
top500_act = [row.ACT_NM for row in tmp_list][:500]
act_dur = df.groupby(["CUS_ID"]).pivot("ACT_NM", top500_act).agg(F.sum("ST_TIME"))

### Join grouped tables

In [55]:
t1 = pageview_sum.alias('pv_sum')
t2 = pageview_avg.alias('pv_avg')

t1.join(t2, 'CUS_ID').show() 

+------+-------+------------------+
|CUS_ID| pg_sum|            pg_avg|
+------+-------+------------------+
|   148|28262.0|3.6533092037228543|
|   463|21018.0|7.6152173913043475|
|   471| 6183.0| 4.857030636292223|
|   496|12942.0| 5.113393915448439|
|   833|30720.0| 7.641791044776119|
|  1088| 9471.0| 3.655345426476264|
|  1238| 7967.0| 5.156634304207119|
|  1342|29545.0| 5.223656294200849|
|  1580|13739.0| 3.329052580566998|
|  1591| 8637.0| 4.686380900705371|
|  1645| 7985.0| 4.615606936416185|
|  1829|57972.0| 5.352908587257618|
|  1959|58172.0| 12.06387391124015|
|  2122|10682.0|  6.25776215582894|
|  2142|12048.0|4.3322545846817695|
|  2366| 9126.0|3.3053241579137995|
|   243|35911.0| 5.872608340147179|
|   392| 9713.0| 5.114797261716693|
|   540|30860.0| 5.068155690589588|
|   623|27741.0| 7.661143330571665|
+------+-------+------------------+
only showing top 20 rows

